## strlen

- length of a string is just an array of chars, ending with last element '/0'

In [11]:
%%file week3.c

#include <stdio.h>

// Linear strlen:
int my_strlen(char *s) {
    int cur_i = 0;
    while (s[cur_i] != '\0') {
        cur_i++;
    }

    // for(cur_i = 0; s[cur_i] != '\0'; cur_i++);

    return cur_i;
}

// Recursive strlen:
int recursive_strlen(char *s) {
    //Base case:
    if (s[0] == '\0') {
        return 0; //Empty string
    }
    // Recursive step
    else {
        return 1 + recursive_strlen(s+1); //s+1 points to the 1st address in the char array/string
    }
}

int main() {
    char s[] = "abc";
    printf("%d\n", my_strlen(s));
    printf("%d\n", recursive_strlen(s));
}

Overwriting lec7.c


In [12]:
%%bash
gcc week3.c -g -o week3
./week3

3
3


## sizeof

- sizeof(int) --> usually 4 bytes
- sizeof(char) --> always 1 B
- sizeof(char *) --> usually addresses take 8B. NOT len of arr/str
- sizeof(int *) --> usually addresses take 8B. NOT len of arr/str

In [ ]:
# Size of an array:

%%file week3.c

int arr[] = {1, 2, 3}
int len = sizeof(arr) / sizeof(arr[0])

However, note that if arr is passed into a function, it is converted to a pointer:

In [ ]:
%%file week3.c

void sz(int *a) {
    sizeof(a); //8
}

int main() {
    int a[] = {1,2,3}
    sz(a);
}

### Similarly, you can get the sizeof a structure:

In [27]:
%%file week3.c
#include <stdio.h>

typedef struct student {
    char name[200];
    double GPA;
} student;

typedef struct student1 {
    char name[201];
    double GPA;
} student1;

int main() {
    printf("%zu\n%zu\n", sizeof(student), sizeof(student1)); //%zu = unsigned type
    student s = {"Alice", 3.9};
    student1 s1 = {"Bob", 3.93};
    return 0;
}

Overwriting lec7.c


In [28]:
%%bash
gcc week3.c -g -o week3
./week3

208
216


## malloc

- Local arrays disappear once a function has finished running
- Arrays in C are not resizable
- malloc allocates space in the memory table to store a block of values

In [ ]:
%%file week3.c

#include <stdlib.h>
#include <stdio.h>

int main() {
    // Allocates space for 150 integers
    // malloc returns the addr of element 0
    // cast the address to int *
    
    int *block_int = (int *)malloc(sizeof(int) * 150);

    //These are the same:

    block_int[7] = 42;
    *(block_int + 7) = 42; 
}

### Ex: Where malloc works/doesn't work

In [ ]:
%%file week3.c

int *make_arr_wrong() {
    int arr[] = {5,6,7};
    return arr;
}

int *make_arr_right() {
    int *arr = (int *)malloc(3*sizeof(int));
    arr[0] = 5;
    arr[1] = 6;
    arr[2] = 7;
    return arr;
}

int main() {
    int *res2 = make_arr_right();
    res2[0] = 123; //Addr of the '5', can be accessed to put 123 in. It exists in memory b/c of malloc.
    int *res1 = make_arr_wrong();
    res1[0] = 123; //This addr is no longer valid because the arr is local
}

## free
- Good practice to free(`block`) memory blocks that you may have allocated
- C cannot use a malloc-ed block for something new until it's freed
- For continuously running programs, you might run out of memory

**Memory Leak**: a situation where memory is allocated but never freed

In [ ]:
%%file week3.c

int *make_arr_wrong() {
    int arr[] = {5,6,7};
    return arr;
}

int *make_arr_right() {
    int *arr = (int *)malloc(3*sizeof(int));
    arr[0] = 5;
    arr[1] = 6;
    arr[2] = 7;
    return arr;
}

int main() {
    int *res2 = make_arr_right();
    res2[0] = 123; //Addr of the '5', can be accessed to put 123 in. It exists in memory b/c of malloc.
    free(res2);
    res2[0]; // Undefined behaviour because block was freed from memory

    int *res1 = make_arr_wrong();
    res1[0] = 123; //This addr is no longer valid because the arr is local
}

#### Block of Structs

In [ ]:
%%file week3.c

#include <stdlib.h>

typedef struct student {
    char name[200];
    int age;
} student;

//Array:
int main() {
    student students[500];
    student *students_block = (student *)malloc(sizeof(student)*500);
}

## Dealing with Strings

- If 'hi' is stored at addrs 1032, 1033, 1034
- The address of the 'h' is 1032, and s1 gets converted to 1032 when used
- s2 = s1 --> allowed, but strings are now aliases. s2 becomes 1032 from 1033

`strcpy(s2, s1)` --> not OK, since cannot copy to addr s2

------------------------------------------------------------

`s2 = (char *)malloc(sizeof(char)*(strlen(s1)+1))` 
`strcpy(s2, s1)` // Copy the contents of s1 into s2


In [ ]:
%%file week3.c

#include <stdio.h>

int main() {
    char s1[] = "hi"; //{'h', 'i', '\0'}
    char s3[5]; //strcpy(s3,s1)
    char *s2 = 0;
}

## Quiz 1: 

In [37]:
%%file week3.c

# include <stdio.h>

int main() {
    int a = 42;

    printf("%d  ", a);

    printf("%ld ", &a);

    printf("%ld ", *a);
    return 0;
}

Overwriting lec7.c


In [ ]:
%%bash
gcc week3.c -g -o week3
./week3

lec7.c:9:20: warning: format specifies type 'long' but the argument has type 'int *' [-Wformat]
    printf("%ld ", &a);
            ~~~    ^~
lec7.c:11:20: error: indirection requires pointer operand ('int' invalid)
    printf("%ld ", *a);
                   ^~
1 warning and 1 error generated.


42  6135147992 

## Pointers to Pointers

- Sometimes, we want to change the value of a pointer inside a function

In [ ]:
%%file week3.c

/*
Set the value at address p_p_a to 0
p_p_a happens to be of type int **
So *p_p_a is of type int*
*/
void set_to_0(int **p_p_a) {
    *p_p_a = 0; 
}

int main() {
    int a = 42;
    int *p_a = &a; //Pointer of a
    set_to_0(&p_a) // &p_a is the pointer of p_a
                   // p_a is now zero, but a is not affected!
}

## Header files

- Can give the compiler instructions for tasks that are performed before compilation
- Basically a class that can be imported and included in C files 
- Standard C library: use <name.h>
- Custom: use "name.h"

## Preprocessor

- Can assign names to certain values
- i.e: `#define PI 3.14`
- Faster than defining a variable

<br>

- Can only define structs once
- Can use an "include guard" in an .h file to avoid defining things twice if the header file is included multiple times

Below:
- Checks whether STUDENT_H (arb name, but usually related to the header file name) is defined
- If it's not defined include the code between #if and #endif

In [6]:
%%file student.h

#if !defined(STUDENT_H)
#define STUDENT_H

typedef struct student {
    char name[20];
    int age;
    double gpa;
} student;

//Can also define a function from student.c here with just the signature:
void hack_gpa(student *s);

#endif


// Second time student.h is #included, nothing will happen
// Name of technique = "include guards"


Overwriting student.h


In [13]:
%%file student.c

#include <stdio.h>
#include "student.h"

void hack_gpa(student *s) { //Can include student because student.h included
    s->gpa = 4.0;
}

int main() {
    student s;
    s.age = 20;
    s.gpa = 3.9;
    hack_gpa(&s);
    printf("%f", s.gpa);
    return 0;
}

Overwriting student.c


In [14]:
%%bash
gcc student.c -g -o student
./student

4.000000

## Strings in Structs

In [ ]:
%%file week3.c
typedef struct student1 {
    char *name; //Store the addr where the name is stored
} student1; //Need to allocate name for each student

typedef struct student2 {
    char name[200]; //Store 200 chars
} student2; //Do not need to allocate name for each student

student1 make_student1(char *name, double gpa) {
    student1 s1;
    
}